In [ ]:
%%bash
mkdir images
cd images
aws s3 cp s3://london-innovation-hub/xray/images . --recursive #copyipynb

___

In [2]:
! pip install tensorflow==2.5.0

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [3]:
%%writefile training.py

import os
os.system('pip install --upgrade pip')
os.system('pip install tensorflow==2.5.0')
os.system('pip install tensorflow-gpu==2.5.0')
os.system('pip install matplotlib')
import argparse
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip
from tensorflow.keras.applications.resnet50 import (
    ResNet50,
    preprocess_input
)


BATCH_SIZE = 32
IMG_SIZE = (1024, 1024)

def data_augmenter():
        data_augmentation = tf.keras.Sequential([
            RandomFlip('horizontal'),
            RandomFlip('vertical')
        ])
        return data_augmentation

def xray_model(image_shape=(1024, 1024), data_augmentation=data_augmenter()):
    input_shape = image_shape + (3,)
    base_model = ResNet50(input_shape=(224,224,3),
                          include_top=False, 
                          weights='imagenet') # From imageNet
    # freeze the base model
    base_model.trainable = False
    # create the input layer
    inputs = tf.keras.Input(shape=input_shape) 
    # apply data augmentation to the inputs
    x = data_augmentation(inputs)
    # data preprocessing using the same weights the model was trained on
    x = preprocess_input(x) 
    # Custom input size with one more Conv layer which outputs 224 * 224 * 3 image
    x = tfl.Conv2D(4, 32, strides=2, activation='relu', input_shape=input_shape, data_format="channels_last"  )(x)
    x = tfl.Conv2D(6, 16, activation='relu', input_shape=(497,497,4), data_format="channels_last"  )(x)
    x = tfl.Conv2D(3, 32, strides=2, activation='relu', input_shape=(482,482,6), data_format="channels_last"  )(x)
    x = tfl.Conv2D(3, 3, activation='relu', input_shape=(226,226,3), data_format="channels_last"  )(x)
    # set training to False to avoid keeping track of statistics in the batch norm layer
    x = base_model(x, training=False) 
    # add the new Binary classification layers
    # use global avg pooling to summarize the info in each channel
    x = tfl.GlobalAveragePooling2D()(x) 
    # include dropout with probability of 0.2 to avoid overfitting
    x = tfl.Dropout(rate=0.2)(x)
    # use a prediction layer with one neuron (as a binary classifier)
    outputs = tfl.Dense(1)(x)
    model = tf.keras.Model(inputs, outputs)
    return model    

def train(agrs):
    model_dir = '/opt/ml/model' # Creates model.tar.gz
    output_dir = '/opt/ml/output/data'
    directory = agrs.train

    train_dataset = image_dataset_from_directory(directory,
                                                 shuffle=True,
                                                 batch_size=BATCH_SIZE,
                                                 image_size=IMG_SIZE,
                                                 validation_split=0.2,
                                                 subset='training',
                                                 seed=42)
    validation_dataset = image_dataset_from_directory(directory,
                                                 shuffle=True,
                                                 batch_size=BATCH_SIZE,
                                                 image_size=IMG_SIZE,
                                                 validation_split=0.2,
                                                 subset='validation',
                                                 seed=42)
    
    AUTOTUNE = tf.data.experimental.AUTOTUNE
    train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)

    data_augmentation = data_augmenter()

    model = xray_model(IMG_SIZE, data_augmentation)

    checkpoint_path = output_dir+'/checkpoint'
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    base_learning_rate = 0.001
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    # Number of Initial Epochs to train on before finetuning              
    initial_epochs = 15

    # Train the model with the callback
    history = model.fit(train_dataset, validation_data=validation_dataset, epochs=initial_epochs, callbacks=[cp_callback])

    # Plot Graph for Initial Training
    acc = [0.] + history.history['accuracy']
    val_acc = [0.] + history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy before Fine tuning')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title('Training and Validation Loss before Fine tuning')
    plt.xlabel('epoch')
    image_path = output_dir+'/plots'
    image_dir = os.path.dirname(image_path)
    plt.savefig(f'{image_dir}/Acuracy_and_Loss_before_finetune.png', bbox_inches='tight')

    # Fine Tuning 

    base_model = model.layers[8]
    base_model.trainable = True
    fine_tune_at = 120

    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = True
    
    # Define a BinaryCrossentropy loss function. Use from_logits=True
    loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    # Define an Adam optimizer with a learning rate of 0.1 * base_learning_rate
    optimizer = tf.keras.optimizers.Adam(learning_rate=base_learning_rate*0.1)
    # Use accuracy as evaluation metric
    metrics=['accuracy']

    model.compile(loss=loss_function,
                optimizer = optimizer,
                metrics=metrics)
    
    fine_tune_epochs = 15
    total_epochs =  initial_epochs + fine_tune_epochs

    history_fine = model.fit(train_dataset,
                            epochs=total_epochs,
                            initial_epoch=history.epoch[-1],
                            validation_data=validation_dataset, 
                            callbacks=[cp_callback])
    
    # Plot for Fine Tuning 
    acc += history_fine.history['accuracy']
    val_acc += history_fine.history['val_accuracy']

    loss += history_fine.history['loss']
    val_loss += history_fine.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.ylim([0, 1])
    plt.plot([initial_epochs-1,initial_epochs-1],
            plt.ylim(), label='Start Fine Tuning')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy with Fine Tuning')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.ylim([0, 1.0])
    plt.plot([initial_epochs-1,initial_epochs-1],
            plt.ylim(), label='Start Tuning later layers')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss with Fine Tuning')
    plt.xlabel('epoch')
    plt.savefig(f'{image_dir}/Acuracy_and_Loss_with_finetune.png', bbox_inches='tight')

    model.save(f"{model_dir}/000000001")


def model_fn():
    # sagemaker inference runtime model load
    model_dir = '/opt/ml/model'
    model = tf.keras.models.load_model(f"{model_dir}/000000001")
    return model
    
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    # Parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    # Outputs by models to be stored directory /opt/ml/output/data
    parser.add_argument('--output_data_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    # Model directory: we will use the default set by SageMaker, /opt/ml/model
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    
    agrs, _ = parser.parse_known_args()
    
    train(agrs)


Overwriting training.py


In [4]:
from sagemaker.tensorflow import TensorFlow
import sagemaker
import time

role = sagemaker.get_execution_role()
bucket_name = 'london-innovation-hub'
bucket_prefix = 'train90322'

In [ ]:
tf_estimator = TensorFlow(
    entry_point="training.py",
    role=role,
    model_dir=f's3://{bucket_name}/{bucket_prefix}/model_dir',
    output_path=f's3://{bucket_name}/{bucket_prefix}/output_data_dir',
    instance_count=1,
    instance_type="ml.g4dn.2xlarge",
    framework_version="2.2",
    py_version="py37",
)
tf_estimator.fit(
    {
        "train": f"s3://london-innovation-hub/xray/full-dataset/"
    }
)

2022-03-09 12:14:59 Starting - Starting the training job...
2022-03-09 12:15:24 Starting - Preparing the instances for trainingProfilerReport-1646828099: InProgress
......
2022-03-09 12:16:26 Downloading - Downloading input data.........
2022-03-09 12:17:44 Training - Downloading the training image......
2022-03-09 12:18:45 Training - Training image download completed. Training in progress.2022-03-09 12:18:45.090158: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2022-03-09 12:18:45.092979: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2022-03-09 12:18:45.186212: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2022-03-09 12:18:47,289 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
2022-03-09 12:1

# Reatime Endpoint

In [ ]:
tf_endpoint_name = 'tf-realtime-inference-endpoint'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
tf_rt_endpoint = tf_estimator.deploy(initial_instance_count=1,instance_type='ml.m5.4xlarge',
                                   endpoint_name=tf_endpoint_name)

In [ ]:
import PIL.Image
import numpy as np
test_image = PIL.Image.open('./data/ok/2021-11-02 08-04-17 - recRTBJBHEMA3AK1 - 8230369205_Az0_0.jpeg')
payload = np.array([np.asarray(test_image.resize((1024, 1024)))])

In [ ]:
test_image.size, payload.shape

In [ ]:
result = tf_rt_endpoint.predict(payload)
result

In [ ]:
tf_rt_endpoint.delete_endpoint()

# Running the prediction from the trained model file directly

In [ ]:
!aws s3 cp s3://bucket-sushant/apollo_tyres/output_data_dir/tensorflow-training-2022-02-06-14-34-10-901/output/model.tar.gz ./saved_model/

In [ ]:
!tar -xzvf ./saved_model/model.tar.gz --directory ./saved_model/

In [ ]:
import tensorflow as tf
model_from_asset = tf.keras.models.load_model('./saved_model/000000001')

In [ ]:
result = model_from_asset.predict(payload)
result